In [20]:

from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np 
import torch
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#import shutil
#shutil.move("/content/gdrive/MyDrive/workspace", "/content")

In [2]:
#!pip install -U jina
#!pip install torch==1.9.0
#!pip install transformers==4.5.1
#!pip install sentence_transformers
#!pip install https://download.pytorch.org/whl/torch_stable.html



In [21]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v1')

In [23]:
df = pd.read_csv('workspace/final.csv',error_bad_lines=False)
df.columns=['Text','Speaker','Episode','Season','Show']
df.head()

,Text,Speaker,Episode,Season,Show
0,Jk2write,Editor,"Episode-02-Pie Hole, Herb",Season-06,Two-And-Half-Men
1,"Thanks, Mom.",NaN,"Episode-02-Pie Hole, Herb",Season-06,Two-And-Half-Men
2,This is a perfect first job for him.,NaN,"Episode-02-Pie Hole, Herb",Season-06,Two-And-Half-Men
3,First job?,NaN,"Episode-02-Pie Hole, Herb",Season-06,Two-And-Half-Men
4,I think you're looking at,NaN,"Episode-02-Pie Hole, Herb",Season-06,Two-And-Half-Men


In [24]:
df.dropna(subset=['Text'],inplace=True)
df.reset_index(drop=True,inplace=True)

# New section

In [25]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    text = df.iloc[ind, 0]
    speaker = df.iloc[ind, 1]
    episode= df.iloc[ind, 2]
    season= df.iloc[ind, 3]
    show= df.iloc[ind, 4]
    doc = Document(text = text)
    doc.tags['speaker'] = speaker
    doc.tags['episode'] = episode
    doc.tags['season'] = season
    doc.tags['show'] = show
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray (length=199574) at 140144259354320>

In [26]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
        
    )
    .add(
        name="text_indexer",
        uses='jinahub://SimpleIndexer',
        #uses_with={'device': 'cuda'},
        
    )
)

In [ ]:
#docs=docs[:1000]

In [27]:

docs[0]

In [28]:
print(docs[4].json())

{
  "id": "145ee868-3efb-11ec-9f85-0242ac1c0002",
  "mime_type": "text/plain",
  "tags": {
    "episode": "Episode-02-Pie Hole, Herb",
    "season": "Season-06",
    "show": "Two-And-Half-Men",
    "speaker": "NaN"
  },
  "text": "I think you're looking at"
}


In [29]:
len(docs)

199574

In [33]:
!rm -rf workspace

with flow:
    flow.index(
      inputs=docs,
        
        parameters = {'name' : 'somethign', 'xyz' : 'fsdfsdfsa'},
        gpus='all'
  )

In [44]:

def query_text():
    query = Document(text = input("Enter the Text - "))
    print()
    with flow:
        response = flow.search(inputs = query,parameters={'limit':20},uses_with ={'device':'cpu'} ,return_results = True)
    matches = response[0].docs[0].matches

    for i in matches[:5]:
        print()
        print(str(i.text),"\n")
        print("series :", i.tags['show'],"\n")
        print("season :", i.tags['season'],"\n")
        print("episode :", i.tags['episode'],"\n")
        print("speaker :", i.tags['speaker'],"\n")
    
        print("_________________________________________________________________________________________________________________-\n")


In [47]:
query_text()

KeyboardInterrupt: ignored

___________________

In [ ]:
!